<a href="https://colab.research.google.com/github/SithuniNudara/CNN/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Convolutional Neural Networks (CNN)**

In [ ]:
!pip install tensorflow

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import zipfile

In [11]:
#Specify the path to the uploaded zip file
zip_file_path = '/content/drive/MyDrive/DataSet (2).zip'
#Specify the directory where you want to extract the file
extraction_path = 'data2'

#Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_path)

In [ ]:
train_path = '/content/data2/DataSet/PetImages/Testing_Data'
test_path = '/content/data2/Dataset/PetImages/Testing_Data'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
#cnn model convolutional nerual network
model = tf.keras.models.Sequential({
    #Note the input shape is the desired size of the image 300x300 with 3 bytes color \n
    #This is th efirst Convolutional\n
    tf.keras.layers.Conv2D(filters=16, kernel_size =3, activation = 'relu',input_shape = (300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    #The second convolutional\n
    tf.keras.layers.Conv2D(filters=16, kernel_size =3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    #The Third convolutional\n
    tf.keras.layers.Conv2D(filters=32, kernel_size =3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    #The Fourth convolutional\n
    tf.keras.layers.Conv2D(filters=64, kernel_size =3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    #The Fifth convolutional\n
    tf.keras.layers.Conv2D(filters=64, kernel_size =3, activation = 'relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    #Flattern the results to feed into a DNN\n
    tf.keras.layers.Flatten(),
    #512 neuron hidden layer\n
    tf.keras.layers.Dense(512, activation = 'relu'),
    #Only 1 output neuron. It will contian a value from 0-1 where 0 fro 1class ('Cat) and 1 fro the other ('Dog)\n,
    tf.keras.layers.Dense(1, activation = 'sigmoid') #family mmebers - 4 , activatioon- softmax

})

model.summary()

In [ ]:
#Compli the model

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Train The Model From Generators

In [ ]:
#All train images will be rescales by 1./255
train_datagen = ImageDataGenerator(rescale=1/255.0)

#Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory('/content/data2/DataSet/PetImages/Training_Data',
                                                    target_size=(300,300),
                                                    batch_size=32,
                                                    class_mode='binary')

#Al test images will be rescaled by 1./255
test_datagen = ImageDataGenerator(rescale=1/255)
#apply predefined specification to test dataset
test_set = test_datagen.flow_from_directory('/content/data2/DataSet/PetImages/Testing_Data',
                                            target_size=(300,300),
                                            batch_size=32,
                                            class_mode='binary')

Train

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_set,
    steps_per_epoch=8,
    epochs=10
)

In [ ]:
#save weights fro future predictions
model.save_weights('first_try.weights.h5')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

# load the saved model weights
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.load_weights('/content/first_try_weights.h5')

# Function to preprocess the image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(300, 300))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.
    return img_array

# Example usage:
image_path='/content/data2/Cat&Dog Dataset new/Training_data/dogs/dog.210.jpg'  # Replace with the actual path to your image
preprocessed_image = preprocess_image(image_path)
prediction = model.predict(preprocessed_image)

# Interpret the prediction
if prediction[0][0] > 0.61:
    print("Prediction: Dog")
else:
    print("Prediction: Cat")
print(prediction)

In [ ]:
#plot variation in loss and accuracy
import matplotlib.pyplot as plt
# plot the loss
plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')



In [ ]:
# plot the accuracy\n",
plt.plot(history.history['accuracy'], label='train acc')
plt.plot(history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')